In [1]:
import os
cache_dir = '/scratch3/workspace/wenlongzhao_umass_edu-reason/dev_kedar/transformers_cache'
os.environ['TRANSFORMERS_CACHE'] = cache_dir
os.environ['HF_HOME']=cache_dir
os.environ['HF_HUB_CACHE']=cache_dir+'/hub'

In [2]:
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, AutoConfig

from datasets import load_from_disk




from vllm.inputs import TokensPrompt
from vllm import LLM, SamplingParams
from tqdm import tqdm
# from small_llm_reasoning.evaluation.gsm8k import get_score, eight_shot_messages
# from small_llm_reasoning.generation.vllm_generation import llama_forward



/work/pi_mccallum_umass_edu/kchimmad_umass_edu/conda_envs/reason/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/work/pi_mccallum_umass_edu/kchimmad_umass_edu/conda_envs/reason/lib/python3.12/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2025-03-17 15:06:25,738	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
# Loading data
data_path= "../datasets/gsm8k"
split='feedback'
data = load_from_disk(f"{data_path}/tokenized/LLaMA3B-Instruct/{split}/zero-shot/")
data

Dataset({
    features: ['question', 'answer', 'input_ids'],
    num_rows: 1000
})

In [4]:
# from huggingface_hub import hf_hub_download
# # import os

# print("HF_HOME:", os.getenv("HF_HOME"))
# print("TRANSFORMERS_CACHE:", os.getenv("TRANSFORMERS_CACHE"))
# print("Hugging Face Cache Directory:", hf_hub_download.__defaults__)


In [5]:
# Loading model
hf_token = os.getenv("hf_token")

# model_name= "meta-llama/Llama-3.2-1B-Instruct"
model_name= "meta-llama/Llama-3.2-3B-Instruct"
# model_name= "meta-llama/Llama-3.1-8B-Instruct"
# model_name= "meta-llama/Llama-3.3-70B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token, cache_dir=cache_dir)
# tokenizer.pad_token_id = tokenizer.eos_token_id



In [6]:
kwargs={
    'download_dir':cache_dir,
    'max_model_len': 2000
}

model = LLM(
        model=model_name,  
        tensor_parallel_size=1,
        trust_remote_code=True,
    **kwargs
) 



INFO 03-17 15:06:42 llm_engine.py:213] Initializing an LLM engine (v0.6.0) with config: model='meta-llama/Llama-3.2-3B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.2-3B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=2000, download_dir='/scratch3/workspace/wenlongzhao_umass_edu-reason/dev_kedar/transformers_cache', load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=meta-llama/Llama-3.2-3B-Instruct, 

Ignored error while writing commit hash to /scratch3/workspace/wenlongzhao_umass_edu-reason/dev_kedar/transformers_cache/models--meta-llama--Llama-3.2-3B-Instruct/refs/main: [Errno 13] Permission denied: '/scratch3/workspace/wenlongzhao_umass_edu-reason/dev_kedar/transformers_cache/models--meta-llama--Llama-3.2-3B-Instruct/refs/main'.
Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:00<00:00,  1.30it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:01<00:00,  1.84it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:01<00:00,  1.73it/s]


INFO 03-17 15:06:44 model_runner.py:926] Loading model weights took 6.0160 GB


INFO 03-17 15:06:45 gpu_executor.py:122] # GPU blocks: 19048, # CPU blocks: 2340
INFO 03-17 15:06:47 model_runner.py:1217] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 03-17 15:06:47 model_runner.py:1221] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 03-17 15:06:58 model_runner.py:1335] Graph capturing finished in 10 secs.


In [60]:
stop_strings =  ['<|eot_id|>', '<|start_header_id|>user<|end_header_id|>', 'Q:', '</s>', '<|im_end|>']


sampling_params = SamplingParams(n=1,
                                 temperature=0,
                                 max_tokens=500,
                                 stop=stop_strings,
                                 logprobs=2
                                )

In [61]:
# for i in tqdm(range(1000)):
#     assert len(data['input_ids'][i]['prompt_token_ids'])>0
    

In [62]:
all_outputs = model.generate(data['input_ids'][143], sampling_params)
# outputs = model.generate(['what is the capital of India','Where is Delhi'], sampling_params)

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.66s/it, est. speed input: 82.48 toks/s, output: 95.72 toks/s]


In [44]:
stop_strings =  ['<|eot_id|>', '<|start_header_id|>user<|end_header_id|>', 'Q:', '</s>', '<|im_end|>']


sampling_params = SamplingParams(n=1,
                                 temperature=0,
                                 max_tokens=500,
                                 stop=stop_strings,
                                 logprobs=2
                                )

In [45]:
all_outputs = model.generate(data['input_ids'], sampling_params)

Processed prompts: 100%|██████████| 1000/1000 [00:33<00:00, 30.21it/s, est. speed input: 3979.79 toks/s, output: 5209.60 toks/s]


In [46]:
# generated_outputs=[]
# for out in all_outputs:
#     for o in out.outputs:
#         # print(o.text)
#         # print(len(o.token_ids))
#         # print(o.token_ids)
#         # print(type(o.logprobs))
#         # print(len(o.logprobs))
#         idx=0
#         for token_log_probs in o.logprobs:
#             assert len(token_log_probs.keys())==2
#             assert list(token_log_probs.keys())[0]==o.token_ids[idx]
#             print(o.token_ids[idx])
#             print(token_log_probs)
#             generated_outputs.append(o.token_ids[idx])
#             idx+=1
#             # print('\n\n')
#             # print(len(token_log_probs.keys()))
#     break


In [47]:
generated_outputs=[]
for out in all_outputs:
    output={}
    output['input']= tokenizer.decode(out.prompt_token_ids, skip_special_tokens=False)
    output['output']=[]
    output['token_ids']=[]
    output['log_probs']=[]
    output['all_returned_log_probs']=[]
    for ith_output in out.outputs:
        idx=0
        output['output'].append(ith_output.text)
        output['token_ids'].append(list(ith_output.token_ids))
        log_probs=[]
        for token_log_probs in ith_output.logprobs:
            log_probs.append(token_log_probs[ith_output.token_ids[idx]].logprob)
            returned_log_probs=[]
            for token, logprob in token_log_probs.items():
                returned_log_probs.append(
                    {
                        'token_id':token,
                        'logprob':logprob.logprob,
                        'rank':logprob.rank,
                        'decoded_token':logprob.decoded_token
                    }
                )
            output['all_returned_log_probs'].append(returned_log_probs)
            idx+=1
        output['log_probs'].append(log_probs)
    generated_outputs.append(output)

In [52]:
for i in range(len(generated_outputs)):
    try:
        assert len(generated_outputs[i]['log_probs'][0])==len(generated_outputs[i]['all_returned_log_probs'])
        assert len(generated_outputs[i]['log_probs'][0])==len(generated_outputs[i]['token_ids'][0])
    except:
        print(f'Index: {i}')
        print(f'log_probs: {len(generated_outputs[i]['log_probs'][0])}')
        print(f'all_returned_log_probs: {len(generated_outputs[i]['all_returned_log_probs'])}')
        print(f'token_ids: {len(generated_outputs[i]['token_ids'][0])}')
    

In [38]:
len(generated_outputs[0]['log_probs'][0])

159

In [39]:
len(generated_outputs[0]['all_returned_log_probs'])

159

In [41]:
len(generated_outputs[0]['token_ids'][0])

159

In [13]:
log_probs=True
generated_outputs = [
        {
            'input': tokenizer.decode(out.prompt_token_ids, skip_special_tokens=False),
            'output': [ith_output.text for ith_output in out.outputs],
            **({
                'log_probs': [
                    [token_log_probs[token_id].logprob for token_log_probs, token_id 
                    in zip(ith_output.logprobs, ith_output.token_ids)]
                    for ith_output in out.outputs
                ]
            } if log_probs else {})
        }
        for out in all_outputs
    ]

In [14]:
len(generated_outputs)

1000

In [15]:
import pandas as pd
df = pd.DataFrame(generated_outputs)

In [16]:
df

,input,output,log_probs
0,<|begin_of_text|><|start_header_id|>system<|en...,[To find out how many building blocks can fit ...,"[[-4.458328112377785e-05, -0.13078656792640686..."
1,<|begin_of_text|><|start_header_id|>system<|en...,"[To find out how much Rachel should budget, we...","[[-0.0030838334932923317, -0.5552582740783691,..."
2,<|begin_of_text|><|start_header_id|>system<|en...,[To find out how many kilograms of meat are le...,"[[-0.016634341329336166, -0.3490457832813263, ..."
3,<|begin_of_text|><|start_header_id|>system<|en...,"[To solve this problem, we need to find out ho...","[[-0.0503850020468235, -0.5876595973968506, -0..."
4,<|begin_of_text|><|start_header_id|>system<|en...,"[To find the total deposit amount, we need to ...","[[-0.026005588471889496, -0.5923835635185242, ..."
...,...,...,...
995,<|begin_of_text|><|start_header_id|>system<|en...,[To find the total number of crates of eggs Ga...,"[[-0.05552699416875839, -0.5832895040512085, -..."
996,<|begin_of_text|><|start_header_id|>system<|en...,[To find the total number of passengers on the...,"[[-0.03839171305298805, -0.40530502796173096, ..."
997,<|begin_of_text|><|start_header_id|>system<|en...,[To find the number of counters and marbles Re...,"[[-0.02048187144100666, -0.6049375534057617, -..."
998,<|begin_of_text|><|start_header_id|>system<|en...,"[To solve this problem, we need to find out ho...","[[-0.010952593758702278, -0.9995432496070862, ..."


In [23]:
len(df.iloc[0]['log_probs'][0])

172

In [25]:
df.iloc[0]['output'][0]

'To find out how many building blocks can fit into the box, we need to divide the volume of the box by the volume of a single building block.\n\nThe volume of the box is calculated by multiplying its height, width, and length:\nVolume of box = height * width * length = 8 * 10 * 12 = 960 cubic inches\n\nThe volume of a single building block is calculated by multiplying its height, width, and length:\nVolume of building block = height * width * length = 3 * 2 * 4 = 24 cubic inches\n\nNow, we divide the volume of the box by the volume of a single building block to find out how many blocks can fit:\nNumber of blocks = Volume of box / Volume of building block = 960 / 24 = 40\n\nThe final answer is 40.'